# Making slides with the Assistants API and DALL-E3

This notebook demonstrates how to leverage DALL-E3 and GPT-4 to create informative and visually appealing slides. <br>
Creating slides is an important component of many jobs, but can be time-consuming. Here, we will show how to use GPT-4 and DALL-E3 to help you create and improve your slides.


In [1]:
from IPython.display import display, Image, Audio
import base64
import time
from openai import OpenAI
import os
import requests
import pandas as pd

client = OpenAI()


Once we have the video frames we craft our prompt and send a request to GPT (Note that we don't need to send every frame for GPT to understand what's going on):


## 0. Setup

In [34]:
import json

#Lets import some functions from https://cookbook.openai.com/examples/assistants_api_overview_python
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content=user_message
)
    return client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)

def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id)

# assistant_id = "asst_ITGfPF8ySQK2qUErH8YMIsop"


## 1. Creating the content

Let's say we are creating a presentation for a quarterly financial review of our company, NotReal Corporation.
We want to use the following data (*note: this dummy data was created using GPT-4/code interpreter*)

Let's load in the data, and take a look

In [3]:
financial_data = pd.read_json('data/NotRealCorp_financial_data.jsonl',lines=True)
financial_data.head()


,Year,Quarter,Distribution channel,Revenue ($M),Costs ($M),Customer count,Time
0,2021,Q1,Online Sales,1.50,1.301953,150,2021 Q1
1,2021,Q1,Direct Sales,1.50,1.380809,151,2021 Q1
2,2021,Q1,Retail Partners,1.50,1.348246,152,2021 Q1
3,2021,Q2,Online Sales,1.52,1.308608,152,2021 Q2
4,2021,Q2,Direct Sales,1.52,1.413305,153,2021 Q2


As you can see, this data has quarterly revenue, costs and customer data across different distribution channels. Let's create an assistant
that can act as a personal analyst and make a nice visualization for out PowerPoint!

In [9]:
#Write JSON version
financial_data.to_json('data/NotRealCorp_financial_data.json')


In [ ]:
# file = client.files.create(
#   file=open('data/NotRealCorp_financial_data.json',"rb"),
#   purpose='assistants',
# )


In [10]:
# assistant = client.beta.assistants.create(
#   instructions="You are a data scientist assistant. When given data and a query, write the proper code and create the proper visualization",
#   model="gpt-4-1106-preview",
#   tools=[{"type": "code_interpreter"}],
#   file_ids=[file.id]
# )


In [13]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Calculate profit (revenue minus cost) by quarter and year, and visualize as a line plot across the distribution channels, where the colors of each line are significantly different",
      "file_ids": [file_id]
    }
  ]
)


In [15]:
file_id = 'file-5qpq4qe3hS6CINFtWNarDYc8'
assistant_id = "asst_ITGfPF8ySQK2qUErH8YMIsop"
thread_id = "thread_siTiDz1y95uU20h1cKByjKgu"


In [ ]:
# api.openai.com/v1/threads/thread_8ytSSU22msIraTt9STHXIFnt/messages\
#     -H "Content-Type: application/json" \
#   -H "Authorization: Bearer $OPENAI_API_KEY" \
#   -H "OpenAI-Beta: assistants=v1"


In [16]:
run = client.beta.threads.runs.create(
    thread_id = thread_id,
    assistant_id=assistant_id,
)


As the creation of a run is an asynchronous operations, let's use the helper function `wait_on_run` from
  the fantastic Assistants API Overview cookbook (https://cookbook.openai.com/examples/assistants_api_overview_python)

In [17]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run


In [19]:
run = wait_on_run(run,thread)


In [ ]:
run


Let's see the messages the Assistant added.

In [20]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

plot_file_id = messages.data[0].content[0].image_file.file_id
image_path = "data/NotRealCorp_plot.png"


{'data': [{'id': 'msg_aUY9hpdcbeq1X5VkKqfXaWbH',
   'assistant_id': 'asst_ITGfPF8ySQK2qUErH8YMIsop',
   'content': [{'image_file': {'file_id': 'file-10tOO4QmT4ymhNIfqLrh237N'},
     'type': 'image_file'},
    {'text': {'annotations': [],
      'value': 'Here is the line plot visualizing the profit per quarter and year across the different distribution channels. Each distribution channel is represented by a line with a unique color, as requested.\n\nThe plot shows the trend of profit over time for each channel, allowing you to easily compare their performance.'},
     'type': 'text'}],
   'created_at': 1700012299,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_gmBZmly2a6l8FEQ4zqHnixzh',
   'thread_id': 'thread_siTiDz1y95uU20h1cKByjKgu'},
  {'id': 'msg_Tkc9pamwW4U9CEWGg6u7Exkb',
   'assistant_id': 'asst_ITGfPF8ySQK2qUErH8YMIsop',
   'content': [{'text': {'annotations': [],
      'value': 'The profits have been successfully c

In [73]:
def convert_file_to_png(file_id, write_path):
    data = client.files.content(file_id)
    data_bytes = data.read()
    with open(write_path, "wb") as file:
        file.write(data_bytes)



Let's load in the plot!

![Financial data plot](data/NotRealCorp_plot.png)

Nice! So, with just one sentence, we were able to have our assistant use code interpreter to
calculate the profitability, and graph the three lineplots of the various distribution channels.<br><br>
Now we have a nice visual for our slide, but we want some insights to go along with it. Let's use GPT-Vision to help us generate those.

## Generating insights

In [ ]:
company_summary = "NotReal Corp is a prominent hardware company that manufactures and sells processors, graphics cards and other essential computer hardware."


First lets encode the image

In [ ]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Getting the base64 string
base64_image = encode_image(image_path)


In [46]:
# submit_message(assistant_id,thread,"Give me two concise bullets of the \
#       most important takeaways and insights from the plot you just created.\
#              These will be used for a slide deck, so keep them brief and informative,\
#                  but they should be more than just a summary, they should be the\
#                      'so what' behind the data."
# )

response = get_response(thread)
#get the bullet points
bullet_points = response.data[0].content[0].text.value


In [47]:
bullet_points


"- **Seasonal Fluctuations Indicated**: Profit trends exhibit a cyclical pattern within each year, suggesting that certain quarters may consistently outperform or underperform others across the distribution channels. This insight can be leveraged for strategic planning and forecasting, indicating that particular attention should be paid to optimizing operations and marketing efforts during peak and off-peak seasons to maximize profitability.\n\n- **Channel Performance Disparity**: There are evident disparities in profit generation between distribution channels. For example, 'Retail Partners' may represent a more stable and generally higher profit stream, while 'Direct Sales' and 'Online Sales' show more variation and occasionally lag behind. This suggests a need to evaluate the effectiveness of each channel's strategy and possibly redistribute investments to enhance the overall profitability and balance of the revenue mix."

Cool! Now we have some insight to accompany our plot. Let's get a compelling title for the slide.

In [58]:
submit_message(assistant_id,thread,"Given the plot and bullet points you created,\
 come up with a brief title for a slide. It should reflect the insights you came up with."
)
response = get_response(thread)
#get the bullet points
title = response.data[0].content[0].text.value
#title = "Maximizing Returns: Navigating Seasonal Peaks and Channel Dynamics"


Nice, now we have a title, a plot and two bullet points. As a final step, let's have DALL-E3 come up with an image to use as the title slide of the presentation. <br><br>
To do this, we just need to ask our Assistant!

In [60]:
submit_message(assistant_id,thread,"Generate an image that represents our company NotRealCorp and \
the trajectory we are on! It should have an inspirational tone"
)


Run(id='run_1Y5bPVSWrNgUoDAHUG6wnSbv', assistant_id='asst_ITGfPF8ySQK2qUErH8YMIsop', cancelled_at=None, completed_at=None, created_at=1700013279, expires_at=1700013879, failed_at=None, file_ids=['file-lGWbUeGIC5PDxh8SkqSqEDw8'], instructions='You are a data scientist assistant. When given data and a query, write the proper code and create the proper visualization', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_siTiDz1y95uU20h1cKByjKgu', tools=[ToolAssistantToolsCode(type='code_interpreter')])

In [61]:
response = get_response(thread)


In [63]:
messages = client.beta.threads.messages.list(thread_id=thread.id)


In [75]:
title_img_file_id = messages.data[0].content[0].image_file.file_id
title_img_write_path = 'data/NotRealCorp_title_image.png'



In [92]:
messages.data


[ThreadMessage(id='msg_ujCSns8vpJTFpZnXXeCDopx2', assistant_id='asst_ITGfPF8ySQK2qUErH8YMIsop', content=[MessageContentImageFile(image_file=ImageFile(file_id='file-wbr9e8peOrcsUOSANlS6WxnK'), type='image_file'), MessageContentText(text=Text(annotations=[TextAnnotationFilePath(end_index=345, file_path=TextAnnotationFilePathFilePath(file_id='file-17N9sra9Tz5yRt8QdSq5sZtj'), start_index=292, text='sandbox:/mnt/data/notrealcorp_inspirational_image.png', type='file_path')], value='I have created an inspirational image that reflects an upward trajectory and a sense of new beginnings for NotRealCorp, showing a sunrise and an ascending path to symbolize growth and progress.\n\nYou can download the image using the following link:\n\n[Download NotRealCorp Inspirational Image](sandbox:/mnt/data/notrealcorp_inspirational_image.png)'), type='text')], created_at=1700013308, file_ids=['file-17N9sra9Tz5yRt8QdSq5sZtj'], metadata={}, object='thread.message', role='assistant', run_id='run_1Y5bPVSWrNgUoDA

In [89]:
messages.data[0].content[0].image_file.file_ida


'file-wbr9e8peOrcsUOSANlS6WxnK'

In [ ]:
title_img_file_id = messages.data[0].content[0].image_file.file_id
title_img_write_path = 'data/NotRealCorp_title_image.png'



In [97]:
client.files.content('mnt/data/notrealcorp_inspirational_image.png')


NotFoundError: Error code: 404 - {'error': {'message': 'Invalid URL (GET /v1/files/mnt/data/notrealcorp_inspirational_image.png/content)', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [81]:
convert_file_to_png(title_img_file_id,title_img_write_path)


NotFoundError: Error code: 404 - {'error': {'message': 'No such File object: file-wbr9e8peOrcsUOSANlS6WxnK', 'type': 'invalid_request_error', 'param': 'id', 'code': None}}

Once we have the video frames we craft our prompt and send a request to GPT (Note that we don't need to send every frame for GPT to understand what's going on):


In [ ]:
PROMPT_MESSAGES = [
    {
        "role": "user",
        "content": [
            "These are frames from a video that I want to upload. Generate a compelling description that I can upload along with the video.",
            *map(lambda x: {"image": x, "resize": 768}, base64Frames[0::10]),
        ],
    },
]
params = {
    "model": "gpt-4-vision-preview",
    "messages": PROMPT_MESSAGES,
    "api_key": os.environ["OPENAI_API_KEY"],
    "headers": {"Openai-Version": "2020-11-07"},
    "max_tokens": 200,
}

result = openai.ChatCompletion.create(**params)
print(result.choices[0].message.content)


## 2. Generating a voiceover for a video with GPT-4 and the TTS API


Let's create a voiceover for this video in the style of David Attenborough. Using the same video frames we prompt GPT to give us a short script:


In [ ]:
PROMPT_MESSAGES = [
    {
        "role": "user",
        "content": [
            "These are frames of a video. Create a short voiceover script in the style of David Attenborough. Only include the narration.",
            *map(lambda x: {"image": x, "resize": 768}, base64Frames[0::10]),
        ],
    },
]
params = {
    "model": "gpt-4-vision-preview",
    "messages": PROMPT_MESSAGES,
    "api_key": os.environ["OPENAI_API_KEY"],
    "headers": {"Openai-Version": "2020-11-07"},
    "max_tokens": 500,
}

result = openai.ChatCompletion.create(**params)
print(result.choices[0].message.content)


Now we can pass the script to the TTS API where it will generate a mp3 of the voiceover:


In [ ]:
response = requests.post(
    "https://api.openai.com/v1/audio/speech",
    headers={
        "Authorization": f"Bearer {os.environ['OPENAI_API_KEY']}",
    },
    json={
        "model": "tts-1",
        "input": result.choices[0].message.content,
        "voice": "onyx",
    },
)

audio = b""
for chunk in response.iter_content(chunk_size=1024 * 1024):
    audio += chunk
Audio(audio)
